# Experiment Testing Adversaries

To do:

Option 1:
1. Find the training data
2. Load the training data
3. Select trajectories which contain scenario 1. 

The training data is very light on the optimal trajectories! My stupid sampling thing was almost certainly part of the problem. I think we should try removing it and seeing if the model will train. 

Option 2:
1. Synthesize scenario 1 forward passes somehow. 

Fixed Result:

4. Visualize a few. 
5. Write code to add adversaries. 
6. Plot left-right preference before/after. 

## Try to Load the Training Data and use that 

In [9]:
import sys 
import os 

sys.path.append("..")

from src.streamlit_app.environment import get_env_and_dt, preprocess_inputs
from src.environments.registration import register_envs
from src.decision_transformer.offline_dataset import TrajectoryDataset, one_hot_encode_observation
from torch.utils.data import DataLoader

register_envs()

# GET MODEL AND ENVIRONMENT
model_index = {
    # The original post, reproduced with 1/10th of the total training epochs
    "models/MiniGrid-MemoryS7FixedStart-v0/WorkingModel.pt": "MemoryDT",
    "models/MiniGrid-MemoryS7FixedStart-v0/MemoryGatedMLP.pt": "MemoryDTGatedMLP",
    "models/MiniGrid-Dynamic-Obstacles-8x8-v0/ReproduceOriginalPostShort.pt": "DynamicObstaclesDT_reproduction",
}

env, dt = get_env_and_dt("../models/MiniGrid-MemoryS7FixedStart-v0/WorkingModel.pt")

# GET TRAINING DATA 
training_trajectories = "../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11-VariedSamplingStrategies_T1_10k_T5_20k.gz"
dataset = TrajectoryDataset(
    training_trajectories, 
    max_len=9, 
    prob_go_from_end=1,
    preprocess_observations=one_hot_encode_observation)
s, a, r, d, rtg, ti, m = dataset[0]
print(s.shape)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning:

WARN: Overriding environment DynamicObstaclesMultiEnv-v0 already in registry.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning:

WARN: Overriding environment CrossingMultiEnv-v0 already in registry.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning:

WARN: Overriding environment MultiRoomMultiEnv-v0 already in registry.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning:

WARN: Overriding environment Probe1-v0 already in registry.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasi

Ignore T-Lens warnings about folding layer norm into unembed, we just did it manually
torch.Size([9, 7, 7, 20])


In [10]:
s.shape
s[0].shape

torch.Size([7, 7, 20])

In [11]:
import torch
from torch.utils.data import DataLoader
# Assuming necessary imports for your dataset and env

dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

# Create lists to collect batches
obs_list, actions_list, rtg_list, ti_list = [], [], [], []

# Iterate through the dataloader
for i, batch in enumerate(dataloader):
    s, a, r, d, rtg, ti, m = batch
    a[a == -10] = env.action_space.n
    obs, actions, rtg, ti = preprocess_inputs(dt, s, rtg, a, ti)
    
    # Append to lists
    obs_list.append(obs)
    actions_list.append(actions)
    rtg_list.append(rtg)
    ti_list.append(ti)
    
    # Break after collecting 10 batches
    if i == 9:
        break

# Concatenate along the batch dimension (assuming it's the 0-th dimension)
obs_stacked = torch.cat(obs_list, dim=0)
actions_stacked = torch.cat(actions_list, dim=0)
rtg_stacked = torch.cat(rtg_list, dim=0)
ti_stacked = torch.cat(ti_list, dim=0)

# Print shapes
print(obs_stacked.shape)
print(actions_stacked.shape)
print(rtg_stacked.shape)
print(ti_stacked.shape)


torch.Size([2560, 9, 7, 7, 20])
torch.Size([2560, 8])
torch.Size([2560, 9, 1])
torch.Size([2560, 9])


In [12]:
import torch 

starts_with_padding_mask = (actions_stacked[:,:4] == torch.tensor([7,7,7,7])).sum(dim=1) == 4
ends_with_forwards = (actions_stacked[:,-4:] == torch.tensor([2,2,2,2])).sum(dim=1) == 4
print(ends_with_forwards.sum())


tensor(0)


In [13]:
from src.visualization import render_minigrid_observation, render_minigrid_observations
from src.environments.utils import reverse_one_hot
import plotly.express as px
import numpy as np 
import random

# get a random batch index
batch_index = random.randint(0, obs_stacked.shape[0] - 1)
batch_index 

one_hot_s = obs_stacked[batch_index]
original_obs = torch.from_numpy(np.stack([reverse_one_hot(one_hot_s[i]) for i in range(one_hot_s.shape[0])]))
images = render_minigrid_observations(env, original_obs)
px.imshow(images, animation_frame=0).show()

In [14]:
obs, actions, rtg, ti = preprocess_inputs(dt, s, rtg, a, ti)
print(obs.shape)
print(actions.shape)
print(rtg.shape)


torch.Size([256, 9, 7, 7, 20])
torch.Size([256, 8])
torch.Size([256, 9, 1])


In [17]:
obs.shape

torch.Size([256, 9, 7, 7, 20])

In [18]:

tokens = dt.to_tokens(obs, actions.unsqueeze(-1), rtg, ti.unsqueeze(-1))
tokens.shape # now we have the tokens, we can run the model 
x, cache = dt.transformer.run_with_cache(tokens, remove_batch_dim=False)

state_preds, action_preds, reward_preds = dt.get_logits(
    x,
    batch_size=256,
    seq_length=9,
    no_actions=False,  # we always pad now.
)

action_preds

tensor([[[ 0.3753,  0.4382, -0.5738,  ...,  0.0824, -1.0965, -0.4971],
         [ 1.9680,  0.6180, -6.1837,  ...,  1.5152,  0.7305,  0.6445],
         [ 2.0623,  0.6053, -5.8181,  ...,  1.3083,  0.4892,  0.6095],
         ...,
         [ 0.6566,  0.3052, -0.9217,  ...,  0.3654, -1.1099, -0.2242],
         [ 5.7083,  5.6541, -1.9668,  ..., -2.6551, -2.0412, -2.7661],
         [-3.3879, -2.5247,  7.8812,  ..., -0.6280, -0.8122, -1.1731]],

        [[ 0.2570, -0.8382, -0.1530,  ...,  0.3714, -1.2210,  0.3707],
         [-0.0107, -0.8057,  0.0756,  ...,  0.3662, -1.2963,  0.3784],
         [ 0.0835, -0.9086,  0.0112,  ...,  0.3772, -1.2716,  0.5223],
         ...,
         [ 0.5680,  0.4389, -0.4616,  ..., -0.1146, -1.4083, -0.4057],
         [ 1.0993,  0.7998, -0.7998,  ..., -0.3313, -1.3772, -0.6870],
         [-0.5187,  0.2612,  2.3857,  ..., -0.2416, -1.7133, -0.8263]],

        [[-1.0096, -3.6262,  3.0661,  ...,  2.3985, -3.8960,  1.7841],
         [-1.7852, -3.5330,  3.3008,  ...,  2

## Generate the Trajectories Manually

In [19]:
from src.environments.memory import MemoryEnv
from src.decision_transformer.offline_dataset import one_hot_encode_observation
import torch
import numpy as np
import plotly.express as px

env = MemoryEnv(
    size=7, 
    random_direction=False, 
    random_start_pos=False, 
    random_length=False, 
    render_mode="rgb_array")

env.reset()

all_scenarios = []
all_observations = []
all_actions = []

for target_obj in ["key", "ball"]:
    for target_pos in ["top", "bottom"]:
        scenario = f"{target_obj}_{target_pos}"
        observations = [] 
        actions= []

        # generate the new environment 
        env._gen_grid(7,7,target_obj=target_obj, target_pos=target_pos)
        obs = env.gen_obs()['image']
        observations.append(obs)
        for _ in range(4):
            action = 2
            next_obs, reward, next_done, next_truncated, info  = env.step(action)
            observations.append(next_obs['image'])
            actions.append(action)

        all_observations.append(observations)
        all_actions.append(actions)
        all_scenarios.append(scenario)
        


all_observations = torch.from_numpy(np.stack(all_observations))
all_actions = torch.from_numpy(np.stack(all_actions))

print(all_observations.shape)
print(all_actions.shape)

torch.Size([4, 5, 7, 7, 3])
torch.Size([4, 4])


In [20]:
# to view them we can use 
from src.visualization import render_minigrid_observations
import plotly.express as px

images = torch.stack([torch.from_numpy(render_minigrid_observations(env, s)) for s in all_observations])
images.shape

px.imshow(images, animation_frame=1, facet_col=0).show()

In [29]:
# Convert these synthetic trajectories into what we need for the model

from src.decision_transformer.offline_dataset import one_hot_encode_observation


one_hot_s = torch.stack([one_hot_encode_observation(all_observations[i]) for i in range(all_observations.shape[0])])

# pad the first dimension to be 9 with all 0's 
obs = torch.cat([
    torch.zeros(one_hot_s.shape[0], 9 - one_hot_s.shape[1], *one_hot_s.shape[2:]), 
    one_hot_s
    ], dim=1)
obs.shape

actions = torch.stack([all_actions[i] for i in range(all_actions.shape[0])])
# pad it with 7's so that the length is 9 
actions = torch.cat([
    torch.ones(actions.shape[0], 8 - actions.shape[1]) * 7, 
    actions
    ], dim=1).to(int).unsqueeze(-1)
actions.shape

time = torch.cat([
    torch.tensor([0]*5),
    torch.arange(1,5),
    ], dim=0)
# repeat it for each trajectory
time = torch.stack([time] * actions.shape[0]).unsqueeze(-1)

# just set to max for now 
rtg = torch.ones(actions.shape[0], time.shape[1]).unsqueeze(-1) * 0.8920


tokens = dt.to_tokens(
    obs, 
    actions,
    rtg, 
    time,
)
tokens.shape # now we have the tokens, we can run the model 
x, adversarial_cache = dt.transformer.run_with_cache(tokens, remove_batch_dim=False)
state_preds, action_preds, reward_preds = dt.get_logits(
    x,
    batch_size=4,
    seq_length=9,
    no_actions=False,  # we always pad now.
)

action_preds.shape

torch.Size([4, 9, 7])

In [30]:
import pandas as pd
from src.streamlit_app.constants import IDX_TO_ACTION

def preds_to_df(action_preds, all_scenarios):
    import pandas as pd
    from src.streamlit_app.constants import IDX_TO_ACTION
    standard_predictions = pd.DataFrame(action_preds[:,-1,:].detach(), columns=IDX_TO_ACTION.values(), index = all_scenarios)
    # get right - left 
    standard_predictions["right_minus_left"] = standard_predictions["right"] - standard_predictions["left"]
    # wide to long 
    standard_predictions = standard_predictions.reset_index().melt(id_vars="index")
    standard_predictions.columns = ["scenario", "action", "logit"]
    # # filter for action in left or right
    # standard_predictions = standard_predictions[standard_predictions.action.isin(["left", "right"])]
    return standard_predictions

standard_predictions = preds_to_df(action_preds, all_scenarios)

px.bar(standard_predictions,
       x = 'action',
       y='logit',
       color = 'action', 
       facet_col='scenario',
        facet_col_wrap=2,
       template='plotly_dark').show()

In [31]:
from src.streamlit_app.causal_analysis_components import get_updated_obs
from src.visualization import render_minigrid_observations
from src.environments.utils import reverse_one_hot

import numpy as np
all_images = []
for scenario in range(4):

    if scenario < 2:
        channel = 6
    else:
        channel = 5

    obs[scenario][4] = get_updated_obs(obs[scenario], 4, 4, 2, channel) # adds a ball to the 4th timeframe at position 4,5 from top LHS. 
    obs[scenario][4] = get_updated_obs(obs[scenario], 4, 0, 5, channel) # adds a ball to the 4th timeframe at position 4,5 from top LHS. 

    obs[scenario].shape
    original_obs = np.stack([reverse_one_hot(obs[scenario][i]) for i in range(obs.shape[1])])
    images = render_minigrid_observations(env, original_obs)
    all_images.append(images)

all_images = torch.from_numpy(np.stack(all_images))
px.imshow(all_images, animation_frame=1, facet_col=0).show()

In [32]:
adversarial_predictions = preds_to_df(action_preds, all_scenarios)

px.bar(adversarial_predictions,
       x = 'action',
       y='logit',
       color = 'action', 
       facet_col='scenario',
        facet_col_wrap=2,
       template='plotly_dark').show()

In [33]:
standard_predictions['has_adversary'] = False
adversarial_predictions['has_adversary'] = True

all_predictions = pd.concat([standard_predictions, adversarial_predictions])

px.bar(all_predictions,
       color ='has_adversary',
       y='logit',
       x = 'action', 
       facet_col='scenario',
        facet_col_wrap=2,
        # place bars next to each other
        barmode='group',
       template='plotly_dark').show()

To do:

1. Consider better ways of plotting these results. What we want is something that will highlight:
    - The decision preference flips (most of the time) (as tracked by right - left)
    - We might need to make this "logit on correct action" as this will show it more clearly. 
    - 0.892 is much weaker

In [34]:
standard_predictions['adversarial_logit']= adversarial_predictions['logit']
standard_predictions['difference'] = standard_predictions['logit'] - standard_predictions['adversarial_logit']
# make an abs difference column
standard_predictions['abs_difference'] = standard_predictions['difference'].abs()
px.bar(
    standard_predictions.query('action == "right_minus_left"'),
    x = 'scenario',
    y = 'abs_difference',
    color = 'scenario',
    labels={'difference': 'Difference in Logits (pos = right)'},
    template='plotly_dark').show()

In [26]:
tmp = standard_predictions.query('action == "right_minus_left"')
tmp['correct_answer'] = [1,1,0,0]

NameError: name 'standard_predictions' is not defined

In [35]:
df = pd.DataFrame(tmp.logit.values.reshape(2,2))
df.columns = ["Key", "Ball"]
df.index = ["Key-Ball", "Ball-Key"]
px.imshow(
    df,
    color_continuous_midpoint=0,
    color_continuous_scale='RdBu',
    text_auto=True,
    template='plotly_dark').show()

NameError: name 'tmp' is not defined

In [28]:
df = pd.DataFrame(tmp.logit.values.reshape(2,2))
df.columns = ["Key", "Ball"]
df.index = ["Key-Ball", "Ball-Key"]
fig = px.imshow(
    df,
    color_continuous_midpoint=0,
    color_continuous_scale='RdBu',
    text_auto=True,
    template='plotly')

fig.show()

NameError: name 'tmp' is not defined